# DSCI 572 Quiz 1 Review Sheet

### Quiz Overview

Quiz 1 will test your understanding of the content covered thus far in both the lecture and the lab. You can prepare for the quiz by going through the lecture slides and the lab tutorials.   

Questions may be:  
(1) simple multiple choice,  
(2) simple conceptual questions that ask for a very short answer (e.g., what does a function from the code covered do), or  
(3) simple completions you need to provide to a code snippet.  

Overall, the questions are designed to be straightforward. The Quiz will be held via Canvas. It will be available between 7:00am-1:00pm PT (Vancouver time). Once you start the Quiz, you will have a total of 40 mins to complete it. If you have any questions beforehand, please do get in touch over course Slack. We do not anticipate people will have questions during the Quiz time itself, but if you do, please feel free to ask over class Slack as well. We cannot guarantee being available for the whole 7-1 period, but we will try to get back to you as soon as we can. For example, Peter will possibly be available 7:00-9:00am PT and Muhammad and/or Ganesh will possibly be available 9:00-10:30. From 10:30-1:00pm PT, if you take the Quiz during this slot, someone will likely be able to answer questions as well (either Ganesh or Peter). Good luck with Quiz 1, and we wish you the best.


## Conceptual Topics

### Hyper Parameter Optimization

- Grid Search vs. Manual Search vs. Random Search
- Sampling from distributions vs. Setting a Parameter Grid

### Neural Network Concepts
- Feed Forward Neural Networks
- Hidden Units vs. Layers
- Weights and Bias
- Activation functions
 - ReLU
 - Sigmoid
 - Tanh
- Dropout
- Initialization
- Word Embeddings

### Optimizers
- Stochastic Gradient Descent (vs. Gradient Descent)
- Learning Rate

## Coding Related

### TorchText
- Loading and tokenizing corpora
- Bucketiteraor

### Pytorch 
- Permute() vs. View()
- Squeeze() and Unsqueeze()
- Matmul() vs. Mul()
- nn.Sequential() vs. nn.Modulelist()
- Intialization of weights and bias of a model  (e.g. layer_name.bias.data.normal_(0,1) )
- Counting Parameters of model (e.g. loop through model.parameters() adding up param.numel() )

### Model Code

Be familiar with (e.g. T1 and T2) model classes there will be questions about something similar to these (completing the class, questions about what parts of the code do).

#### FFNN Class Code from Lab 1

In [ ]:
class TwoLayerNN(nn.Module): #feel free to change the inputs it takes
    def __init__(self, input_dim, hidden_units=20, output_classes=20, nonlin=nn.ReLU()):  
        super(TwoLayerNN, self).__init__()
        self.dense0 = nn.Linear(input_dim, hidden_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(p=.1)
        self.dense1 = nn.Linear(hidden_units, hidden_units)
        self.output = nn.Linear(hidden_units, output_classes)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

In [ ]:
class MLP_Seq(nn.Module): #feel free to change the inputs it takes
    def __init__(self, input_dim, hidden_units=20, output_classes=20, nonlin=nn.ReLU(), dropout=.5):  
        super(MLP_Seq, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim,hidden_units),
            nonlin,
            nn.Dropout(p=dropout),
            nn.Linear(hidden_units,hidden_units),
            nonlin,
            nn.Dropout(p=dropout),
            nn.Linear(hidden_units,hidden_units),
            nonlin,
            nn.Dropout(p=dropout),
            nn.Linear(hidden_units,hidden_units),
            nonlin,
            nn.Dropout(p=dropout),
            nn.Linear(hidden_units,hidden_units),
            nonlin,
            nn.Dropout(p=dropout),
            nn.Linear(hidden_units,output_classes),
            nn.Softmax(dim=-1)
        )

    def forward(self, X, **kwargs):
        return self.layers(X)

In [ ]:
class MLP_Mod(nn.Module): 
    def __init__(self, input_dim,num_layers=5, hidden_units=20, output_classes=20, nonlin=nn.ReLU(), dropout=.5):  
        super(MLP_Mod, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            if i==0:
                self.layers.append(nn.Linear(input_dim,hidden_units))
            else:
                self.layers.append(nn.Linear(hidden_units,hidden_units))
            self.layers.append(nonlin)
            self.layers.append(nn.Dropout(p=dropout))
        self.layers.append(nn.Linear(hidden_units,output_classes))
        self.layers.append(nn.Softmax(dim=-1))

    def forward(self, X, **kwargs):
        for layer in self.layers:
            X = layer(X)
        return X

#### T2 from Lab 2   

In [ ]:
class DimensionTestModel(nn.Module):
  
  def __init__(self, input_size, filters, hidden, output_size):
    super(DimensionTestModel, self).__init__()
    self.cnn = nn.Conv1d(input_size, filters, kernel_size=3, padding =1) 
    self.activation = nn.ReLU()
    self.rnn = nn.RNN(filters, hidden)  
    self.linear_layer = nn.Linear(hidden, output_size) 
    self.softmax_layer = nn.LogSoftmax(dim=1)
  
  def forward(self, x):   #(Assumes input is [N,L,H])
    x = self.cnn(x.permute(0,2,1))
    x = self.activation(x)
    x, _ = self.rnn(x.permute(2,0,1)) 
    x = self.linear_layer(x.permute(1,0,2))
    x = self.softmax_layer(x)
    return x